In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsapi
import json
import re
import datetime

from tqdm import tqdm
from collections import OrderedDict

%matplotlib inline

In [115]:
def generate_teams_dict():
    team_params = {'activeStatus':'Y', 'season':2019, 'sportIds':1, 'fields':'teams,id,name,teamCode,fileCode,teamName,locationName,shortName,venue'}
    teams_info = statsapi.get('teams', team_params)

    teams_dict = {}
    for team in teams_info['teams']:
        teams_dict[team['fileCode']] = team['id']
    return teams_dict

In [116]:
def get_yesterdays_games():
    yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
    return yesterday.strftime("%m/%d/%Y")

In [117]:
def get_todays_games():
    return statsapi.schedule(datetime.datetime.today().strftime("%m/%d/%Y"))

def matchups_and_prob_pitchers_dicts(day='today'):
    games_today = get_todays_games()
    if day == 'yesterday':
        games_today = get_yesterdays_games()
        
    home_away_dict = {}
    prob_pitchers_dict = {}
    for game in games_today:
        prob_pitchers_dict[game['away_id']] = game['away_probable_pitcher']
        prob_pitchers_dict[game['home_id']] = game['home_probable_pitcher']
        
        home_away_dict[game['home_id']] = game['away_id']
        
    away_home_dict = {v:k for k, v in home_away_dict.items()}
    return home_away_dict, away_home_dict, prob_pitchers_dict

In [118]:
def get_player_list(team_id):
    player_names = []
    roster = statsapi.roster(team_id)
    roster_list = roster.split("\n")[:-1]
    for player in roster_list:
        player_names.append(" ".join(player.split()[2:]))
    return player_names

get_player_list(137)

['Alex Dickerson',
 'Austin Slater',
 'Brandon Belt',
 'Brandon Crawford',
 'Buster Posey',
 'Donovan Solano',
 'Drew Pomeranz',
 'Jeff Samardzija',
 'Joe Panik',
 'Kevin Pillar',
 'Madison Bumgarner',
 'Mark Melancon',
 'Mike Yastrzemski',
 'Pablo Sandoval',
 'Reyes Moronta',
 'Sam Coonrod',
 'Sam Dyson',
 'Shaun Anderson',
 'Stephen Vogt',
 'Tony Watson',
 'Trevor Gott',
 'Tyler Austin',
 'Tyler Beede',
 'Will Smith',
 'Zach Green']

In [119]:
def get_player_id_from_name(player_name):
    try:
        return statsapi.lookup_player(player_name)[0]['id']
    except IndexError:
        return False

def check_pos_player(player_name):
    try:
        return statsapi.lookup_player(player_name)[0]['primaryPosition']['abbreviation'] != "P"
    except IndexError:
        return False

def get_current_season_stats(player_name):

    if not check_pos_player(player_name):
        raise ValueError("Player name entered is not a position player")
        
    player_id = get_player_id_from_name(player_name)
    stats_dict = OrderedDict({"Name": player_name, "ID": player_id, 
                  "Team": statsapi.lookup_player(player_id)[0]['currentTeam']['id']})
    
    # Look up the player's current season hitting stats
    get_player_stats = statsapi.player_stats(player_id, 'hitting') 
    
    # Get the stats for the most recent season
    curr_season_stats = get_player_stats.split("Season Hitting")[-1]
    
    #Break up the stats into a list
    stats_list = curr_season_stats.split("\n")[1:-2]
    for stat in stats_list:
        stat_name = re.search("[A-Za-z]+", stat).group()
        stat_val = re.search("[^:A-Za-z]+", stat).group()
        try:
            stats_dict[stat_name] = float(stat_val)
        except ValueError:
            stats_dict[stat_name] = 0.0
    return stats_dict

In [122]:
# These functions were defined with the help of toddrob99 on github, who developed the
# MLB-StatsAPI module. I made a post on reddit.com/r/mlbdata, which he mantains to 
# answer questions about making API calls for specific purposes. I asked how to get stats
# over the past x days and how to get head-to-head batting stats. The post is linked
# here: https://www.reddit.com/r/mlbdata/comments/cewwfo/getting_headtohead_batting_stats_and_last_x_games/?

def batting_past_N_days(N, player_id, end_date=datetime.datetime.today()):
    
    start_date = (end_date - datetime.timedelta(days=N)).strftime("%m/%d/%Y")
    end_date = end_date.strftime("%m/%d/%Y")
    hydrate = ('stats(group=[hitting],type=[byDateRange],startDate={},endDate={}),currentTeam'.
               format(start_date, end_date))
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    batting_stats = r['people'][0]['stats'][0]['splits'][0]['stat']

    # Only get rate stats for past N days
    filtered = {k + "_p{}d".format(N):float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'} 
    filtered = OrderedDict(sorted(filtered.items()))
    
    return filtered

def get_h2h_vs_pitcher(batter_id, opponent_id):
    
    hydrate = 'stats(group=[hitting],type=[vsPlayer],opposingPlayerId={},season=2019,sportId=1)'.format(opponent_id)
    params = {'personId': batter_id, 'hydrate':hydrate, 'sportId':1}
    r = statsapi.get('person',params)
    
    try: 
        batting_stats = r['people'][0]['stats'][1]['splits'][0]['stat']
    except KeyError:
        return OrderedDict({'atBats_h2h': 0.0, 'avg_h2h': 0.0, 'hits_h2h': 0.0, 
                            'obp_h2h': 0.0, 'ops_h2h': 0.0, 'slg_h2h': 0.0})
    
    # Only get rate stats vs pitcher
    filtered = {k + "_h2h":float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'
                or k == 'atBats'} 
    
    filtered = OrderedDict(sorted(filtered.items()))
    
    return filtered

In [123]:
def pitching_past_N_days(N, player_id, end_date=datetime.datetime.today()):
    
    start_date = (end_date - datetime.timedelta(days=N)).strftime("%m/%d/%Y")
    end_date = end_date.strftime("%m/%d/%Y")
    hydrate = 'stats(group=[pitching],type=[byDateRange],startDate={},endDate={}),currentTeam'.format(start_date, end_date)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    pitching_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}d".format(N):float(v) for k, v in pitching_stats.items() 
                if type(v) == str and v != ".---"} 
    
    return filtered

In [124]:
def check_pitcher_right_handed(pitcher_id):
    try:
        params = {'personId': pitcher_id}
        r = statsapi.get('person',params)
        return r['people'][0]['pitchHand']['code'] == 'R'
    except IndexError:
        return False

In [125]:
def check_batter_right_handed(batter_id):
    try:
        params = {'personId': batter_id}
        r = statsapi.get('person',params)
        return r['people'][0]['batSide']['code'] == 'R'
    except IndexError:
        return False

In [126]:
def check_pitcher_batter_opposite_hand(batter_id, pitcher_id):
    return check_pitcher_right_handed(pitcher_id) != check_batter_right_handed(batter_id)

In [127]:
def player_got_hit_in_game(player_id, game_id, home_or_away):
    
    params = {'gamePk':game_id,
      'fields': 'gameData,teams,teamName,shortName,teamStats,batting,atBats,runs,hits,rbi,strikeOuts,baseOnBalls,leftOnBase,players,boxscoreName,liveData,boxscore,teams,players,id,fullName,batting,avg,ops,era,battingOrder,info,title,fieldList,note,label,value'}
    r = statsapi.get('game', params)
    player_stats = r['liveData']['boxscore']['teams'][home_or_away]['players'].get('ID' + str(player_id), False)
    if not player_stats: 
        return False 
    else:
        return player_stats['stats']['batting'].get('hits', 0) > 0

In [128]:
def convert_to_FL_format(name):
    last_first = name.split(",")
    last_first.reverse()
    last_first[0] = last_first[0].strip()
    return " ".join(last_first)

In [129]:
def get_opposing_pitcher(player_id, game_id):
    teams = statsapi.get('schedule', {'sportId': '1', 'gamePk': game_id, 
                                      'hydrate':'probablePitcher'})['dates'][0]['games'][0]['teams']
    home_team_id = teams['home']['team']['id']
    away_team_id = teams['away']['team']['id']
    
    home_prob_pitcher = teams['home']['probablePitcher']['fullName']
    away_prob_pitcher = teams['away']['probablePitcher']['fullName']
    
    if statsapi.lookup_player(player_id)[0]['currentTeam']['id'] == home_team_id:
        return away_prob_pitcher
    else: 
        return home_prob_pitcher

In [130]:
def batting_past_N_games(N, player_id):
    hydrate = 'stats(group=[hitting],type=[lastXGames],limit={}),currentTeam'.format(N)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    batting_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}G".format(N):float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'} 
    
    filtered = OrderedDict(sorted(filtered.items()))
    
    return filtered

def pitching_past_N_games(N, player_id):
    hydrate = 'stats(group=[pitching],type=[lastXGames],limit={}),currentTeam'.format(N)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    pitching_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {(k + "_p{}G".format(N)):(float(v) if v != '.---' else 0.0) 
                for k, v in pitching_stats.items() 
                if type(v) == str} 
    
    filtered = OrderedDict(sorted(filtered.items()))
    
    return filtered

In [145]:
# This cell generates rows for a DataFrame of batting stats--one row per player

yesterday = (datetime.datetime.today() - datetime.timedelta(days = 1)).strftime("%m/%d/%Y")
today = datetime.datetime.today().strftime("%m/%d/%Y")

###############################################################
# 
# Change GENERATE_TRAIN_DATA to False to generate 
# data for today's games instead, which won't have 
# labels included for whether or not the player
# got a hit
#
GENERATE_TRAIN_DATA = False
#
################################################################

gameday = yesterday
if not GENERATE_TRAIN_DATA:
    gameday = today
    
rows_list = []
for game in tqdm(statsapi.schedule(gameday)):
    
    if game['status'] not in ['In Progress', 'Final']:
        continue
    
    game_id = game['game_id']
    away_id = game['away_id']
    home_id = game['home_id']
    home_player_list = get_player_list(home_id)
    away_player_list = get_player_list(away_id)
    
    away_prob_Pname = convert_to_FL_format(game['away_probable_pitcher'])
    home_prob_Pname = convert_to_FL_format(game['home_probable_pitcher'])
    
    away_probable_pitcher = get_player_id_from_name(away_prob_Pname)
    home_probable_pitcher = get_player_id_from_name(home_prob_Pname)
    
    away_pitcher_p5G = pitching_past_N_games(5, away_probable_pitcher)
    home_pitcher_p5G = pitching_past_N_games(5, home_probable_pitcher)
    
    for player in home_player_list:
#         print(player)
        player_id = get_player_id_from_name(player)
        try:
            new_row = list(get_current_season_stats(player).values())
            new_row += list(batting_past_N_games(7, player_id).values())
            new_row += list(batting_past_N_games(15, player_id).values())
            new_row += list(away_pitcher_p5G.values())
            new_row += list(get_h2h_vs_pitcher(player_id, away_probable_pitcher).values())
            new_row.append(float(check_pitcher_batter_opposite_hand(batter_id=player_id, 
                                                                  pitcher_id=away_probable_pitcher)))
            if GENERATE_TRAIN_DATA:
                new_row.append(player_got_hit_in_game(player_id, game_id, 'home'))
                
            rows_list.append(new_row)
        except (ValueError, IndexError):
            continue

    for player in away_player_list:
#         print(player)
        player_id = get_player_id_from_name(player)
        try:
            new_row = list(get_current_season_stats(player).values())
            new_row += list(batting_past_N_games(7, player_id).values())
            new_row += list(batting_past_N_games(15, player_id).values())
            new_row += list(home_pitcher_p5G.values())
            new_row += list(get_h2h_vs_pitcher(player_id, home_probable_pitcher).values())
            new_row.append(float(check_pitcher_batter_opposite_hand(batter_id=player_id, 
                                                                  pitcher_id=away_probable_pitcher)))
            if GENERATE_TRAIN_DATA:
                new_row.append(player_got_hit_in_game(player_id, game_id, 'away'))
                
            rows_list.append(new_row)
        except (ValueError, IndexError):
            continue



100%|██████████| 9/9 [08:37<00:00, 57.64s/it]

In [148]:
sample_hitter = get_player_id_from_name("Kevin Pillar")
sample_pitcher = get_player_id_from_name("Jacob DeGrom")
player_stats_columns = list(get_current_season_stats("Kevin Pillar").keys())
player_stats_columns += list(batting_past_N_games(7, sample_hitter).keys())
player_stats_columns += list(batting_past_N_games(15, sample_hitter).keys())
player_stats_columns += list(pitching_past_N_games(5, sample_pitcher).keys())
player_stats_columns += list(get_h2h_vs_pitcher(sample_hitter, sample_pitcher).keys())

if GENERATE_TRAIN_DATA:
    player_stats_columns += ['pitcher_hitter_opposite_hand', 'player_got_hit']
else:
    player_stats_columns += ['pitcher_hitter_opposite_hand']

In [149]:
player_stats_table = pd.DataFrame(data=rows_list, columns=player_stats_columns)
player_stats_table

,Name,ID,Team,gamesPlayed,groundOuts,runs,doubles,triples,homeRuns,strikeOuts,...,walksPer9Inn_p5G,whip_p5G,winPercentage_p5G,atBats_h2h,avg_h2h,hits_h2h,obp_h2h,ops_h2h,slg_h2h,pitcher_hitter_opposite_hand
0,Adeiny Hechavarria,588751,121,51.0,39.0,19.0,6.0,0.0,5.0,22.0,...,3.38,1.50,0.00,0.0,0.000,0.0,0.000,0.000,0.000,1.0
1,Amed Rosario,642708,121,99.0,100.0,47.0,20.0,5.0,11.0,85.0,...,3.38,1.50,0.00,4.0,0.250,1.0,0.400,1.150,0.750,1.0
2,Dominic Smith,642086,121,87.0,39.0,34.0,10.0,0.0,10.0,44.0,...,3.38,1.50,0.00,0.0,0.000,0.0,0.000,0.000,0.000,0.0
3,J.D. Davis,605204,121,86.0,68.0,31.0,11.0,1.0,9.0,52.0,...,3.38,1.50,0.00,1.0,1.000,1.0,1.000,3.000,2.000,1.0
4,Jeff McNeil,643446,121,88.0,81.0,50.0,27.0,1.0,9.0,48.0,...,3.38,1.50,0.00,0.0,0.000,0.0,0.000,0.000,0.000,0.0
5,Juan Lagares,501571,121,84.0,45.0,19.0,6.0,0.0,2.0,50.0,...,3.38,1.50,0.00,1.0,0.000,0.0,0.000,0.000,0.000,1.0
6,Michael Conforto,624424,121,93.0,62.0,52.0,18.0,0.0,18.0,93.0,...,3.38,1.50,0.00,4.0,1.000,4.0,1.000,2.750,1.750,0.0
7,Pete Alonso,624413,121,101.0,85.0,64.0,21.0,2.0,33.0,108.0,...,3.38,1.50,0.00,1.0,1.000,1.0,1.000,2.000,1.000,1.0
8,Robinson Cano,429664,121,77.0,94.0,29.0,19.0,0.0,9.0,53.0,...,3.38,1.50,0.00,2.0,0.500,1.0,0.500,1.000,0.500,0.0
9,Todd Frazier,453943,121,78.0,60.0,35.0,8.0,2.0,13.0,68.0,...,3.38,1.50,0.00,1.0,1.000,1.0,1.000,3.000,2.000,1.0


In [150]:
player_stats_table.to_csv("player_stats_{}.csv".format(gameday.replace("/", "_")), index=False)